In [ ]:
#must be very first statement

import tensorflow as tf
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [4]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets import cifar10
import numpy as np
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	#(trainX, _), (_, _) = load_data()
	#filter out non birds
    
	# The data, split between train and test sets:
	(x_train_all, y_train_all), (x_test_all, y_test_all) = cifar10.load_data()

	#temp lists
	x_train = []
	y_train_numerical = []

	#filter training data for my_labels
	for i in range(len(x_train_all)):
		if y_train_all[i] in [2]:
			x_train.append(x_train_all[i])
			#y_train_numerical.append(convert_label(y_train_all[i][0]))
		else:
			pass
	trainX = np.array(x_train)

	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

C:\Users\Admin\Anaconda3\envs\ml20\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Admin\Anaconda3\envs\ml20\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/39, d1=0.688, d2=0.696 g=0.691
>1, 2/39, d1=0.629, d2=0.698 g=0.689
>1, 3/39, d1=0.570, d2=0.703 g=0.684


C:\Users\Admin\Anaconda3\envs\ml20\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 4/39, d1=0.480, d2=0.715 g=0.674
>1, 5/39, d1=0.387, d2=0.740 g=0.657
>1, 6/39, d1=0.283, d2=0.776 g=0.641
>1, 7/39, d1=0.195, d2=0.800 g=0.647
>1, 8/39, d1=0.151, d2=0.774 g=0.696
>1, 9/39, d1=0.108, d2=0.701 g=0.769
>1, 10/39, d1=0.095, d2=0.653 g=0.823
>1, 11/39, d1=0.063, d2=0.656 g=0.858
>1, 12/39, d1=0.065, d2=0.669 g=0.875
>1, 13/39, d1=0.074, d2=0.673 g=0.878
>1, 14/39, d1=0.142, d2=0.678 g=0.876
>1, 15/39, d1=0.146, d2=0.712 g=0.859
>1, 16/39, d1=0.218, d2=0.737 g=0.822
>1, 17/39, d1=0.263, d2=0.824 g=0.786
>1, 18/39, d1=0.177, d2=0.825 g=0.821
>1, 19/39, d1=0.229, d2=0.712 g=0.883
>1, 20/39, d1=0.314, d2=0.622 g=0.910
>1, 21/39, d1=0.264, d2=0.612 g=0.927
>1, 22/39, d1=0.295, d2=0.629 g=0.908
>1, 23/39, d1=0.158, d2=0.652 g=0.884
>1, 24/39, d1=0.192, d2=0.690 g=0.844
>1, 25/39, d1=0.126, d2=0.754 g=0.795
>1, 26/39, d1=0.057, d2=0.800 g=0.823
>1, 27/39, d1=0.114, d2=0.714 g=0.885
>1, 28/39, d1=0.211, d2=0.684 g=0.854
>1, 29/39, d1=0.082, d2=0.639 g=0.889
>1, 30/39, d1=0.11

>6, 27/39, d1=0.586, d2=0.575 g=1.381
>6, 28/39, d1=0.418, d2=0.569 g=1.547
>6, 29/39, d1=0.396, d2=0.457 g=1.726
>6, 30/39, d1=0.395, d2=0.506 g=1.577
>6, 31/39, d1=0.392, d2=0.463 g=1.687
>6, 32/39, d1=0.440, d2=0.417 g=1.728
>6, 33/39, d1=0.500, d2=0.524 g=1.577
>6, 34/39, d1=0.491, d2=0.588 g=1.664
>6, 35/39, d1=0.467, d2=0.577 g=1.533
>6, 36/39, d1=0.601, d2=0.688 g=1.628
>6, 37/39, d1=0.689, d2=0.531 g=1.385
>6, 38/39, d1=0.516, d2=0.642 g=1.507
>6, 39/39, d1=0.539, d2=0.576 g=1.463
>7, 1/39, d1=0.580, d2=0.627 g=1.346
>7, 2/39, d1=0.563, d2=0.500 g=1.434
>7, 3/39, d1=0.576, d2=0.688 g=1.491
>7, 4/39, d1=0.517, d2=0.433 g=1.608
>7, 5/39, d1=0.698, d2=0.699 g=1.207
>7, 6/39, d1=0.544, d2=0.767 g=1.327
>7, 7/39, d1=0.732, d2=0.552 g=1.304
>7, 8/39, d1=0.682, d2=0.725 g=1.227
>7, 9/39, d1=0.527, d2=0.617 g=1.303
>7, 10/39, d1=0.565, d2=0.500 g=1.266
>7, 11/39, d1=0.575, d2=0.534 g=1.181
>7, 12/39, d1=0.448, d2=0.565 g=1.281
>7, 13/39, d1=0.687, d2=0.495 g=1.136
>7, 14/39, d1=0.493, 

...

>179, 4/39, d1=0.649, d2=0.539 g=0.982
>179, 5/39, d1=0.643, d2=0.578 g=0.964
>179, 6/39, d1=0.650, d2=0.552 g=0.929
>179, 7/39, d1=0.626, d2=0.572 g=0.935
>179, 8/39, d1=0.627, d2=0.610 g=0.917
>179, 9/39, d1=0.551, d2=0.606 g=0.976
>179, 10/39, d1=0.620, d2=0.626 g=0.877
>179, 11/39, d1=0.569, d2=0.656 g=0.906
>179, 12/39, d1=0.552, d2=0.608 g=0.967
>179, 13/39, d1=0.641, d2=0.596 g=0.916
>179, 14/39, d1=0.689, d2=0.576 g=1.007
>179, 15/39, d1=0.619, d2=0.576 g=1.059
>179, 16/39, d1=0.602, d2=0.572 g=0.999
>179, 17/39, d1=0.587, d2=0.545 g=1.022
>179, 18/39, d1=0.568, d2=0.543 g=0.965
>179, 19/39, d1=0.650, d2=0.617 g=0.955
>179, 20/39, d1=0.563, d2=0.575 g=0.950
>179, 21/39, d1=0.646, d2=0.611 g=0.877
>179, 22/39, d1=0.663, d2=0.629 g=0.951
>179, 23/39, d1=0.553, d2=0.629 g=0.957
>179, 24/39, d1=0.540, d2=0.677 g=1.009
>179, 25/39, d1=0.644, d2=0.593 g=0.990
>179, 26/39, d1=0.611, d2=0.572 g=0.962
>179, 27/39, d1=0.654, d2=0.626 g=0.959
>179, 28/39, d1=0.587, d2=0.572 g=0.908
>179, 

>184, 15/39, d1=0.525, d2=0.676 g=0.955
>184, 16/39, d1=0.690, d2=0.647 g=0.965
>184, 17/39, d1=0.677, d2=0.625 g=0.993
>184, 18/39, d1=0.690, d2=0.587 g=0.999
>184, 19/39, d1=0.629, d2=0.605 g=1.017
>184, 20/39, d1=0.631, d2=0.553 g=1.057
>184, 21/39, d1=0.678, d2=0.521 g=1.094
>184, 22/39, d1=0.540, d2=0.583 g=0.990
>184, 23/39, d1=0.643, d2=0.673 g=0.924
>184, 24/39, d1=0.640, d2=0.644 g=0.829
>184, 25/39, d1=0.564, d2=0.681 g=0.827
>184, 26/39, d1=0.583, d2=0.638 g=0.852
>184, 27/39, d1=0.607, d2=0.716 g=0.921
>184, 28/39, d1=0.566, d2=0.595 g=0.947
>184, 29/39, d1=0.680, d2=0.590 g=0.987
>184, 30/39, d1=0.689, d2=0.538 g=0.984
>184, 31/39, d1=0.565, d2=0.547 g=0.935
>184, 32/39, d1=0.596, d2=0.600 g=0.951
>184, 33/39, d1=0.585, d2=0.592 g=0.911
>184, 34/39, d1=0.672, d2=0.569 g=0.907
>184, 35/39, d1=0.584, d2=0.605 g=0.949
>184, 36/39, d1=0.551, d2=0.555 g=0.963
>184, 37/39, d1=0.541, d2=0.609 g=0.960
>184, 38/39, d1=0.554, d2=0.602 g=0.978
>184, 39/39, d1=0.548, d2=0.594 g=0.958


>189, 27/39, d1=0.701, d2=0.576 g=0.943
>189, 28/39, d1=0.678, d2=0.581 g=0.875
>189, 29/39, d1=0.630, d2=0.609 g=0.914
>189, 30/39, d1=0.591, d2=0.609 g=0.957
>189, 31/39, d1=0.583, d2=0.589 g=0.956
>189, 32/39, d1=0.638, d2=0.573 g=0.935
>189, 33/39, d1=0.609, d2=0.635 g=0.963
>189, 34/39, d1=0.605, d2=0.582 g=1.017
>189, 35/39, d1=0.638, d2=0.556 g=1.027
>189, 36/39, d1=0.512, d2=0.544 g=0.993
>189, 37/39, d1=0.648, d2=0.655 g=1.004
>189, 38/39, d1=0.696, d2=0.524 g=0.946
>189, 39/39, d1=0.636, d2=0.634 g=0.924
>190, 1/39, d1=0.627, d2=0.710 g=0.943
>190, 2/39, d1=0.644, d2=0.602 g=0.974
>190, 3/39, d1=0.647, d2=0.582 g=0.917
>190, 4/39, d1=0.567, d2=0.545 g=0.972
>190, 5/39, d1=0.667, d2=0.598 g=0.970
>190, 6/39, d1=0.631, d2=0.602 g=1.019
>190, 7/39, d1=0.590, d2=0.549 g=0.996
>190, 8/39, d1=0.617, d2=0.583 g=0.976
>190, 9/39, d1=0.684, d2=0.582 g=1.126
>190, 10/39, d1=0.666, d2=0.541 g=1.015
>190, 11/39, d1=0.661, d2=0.587 g=1.018
>190, 12/39, d1=0.591, d2=0.602 g=0.964
>190, 13/

>194, 39/39, d1=0.628, d2=0.626 g=0.990
>195, 1/39, d1=0.609, d2=0.682 g=0.948
>195, 2/39, d1=0.609, d2=0.528 g=1.021
>195, 3/39, d1=0.583, d2=0.671 g=1.059
>195, 4/39, d1=0.739, d2=0.636 g=0.979
>195, 5/39, d1=0.638, d2=0.586 g=0.959
>195, 6/39, d1=0.592, d2=0.614 g=0.951
>195, 7/39, d1=0.708, d2=0.564 g=0.931
>195, 8/39, d1=0.632, d2=0.601 g=0.923
>195, 9/39, d1=0.670, d2=0.711 g=0.909
>195, 10/39, d1=0.651, d2=0.685 g=1.019
>195, 11/39, d1=0.686, d2=0.610 g=1.043
>195, 12/39, d1=0.665, d2=0.550 g=1.067
>195, 13/39, d1=0.675, d2=0.517 g=1.058
>195, 14/39, d1=0.748, d2=0.592 g=1.031
>195, 15/39, d1=0.659, d2=0.579 g=0.989
>195, 16/39, d1=0.674, d2=0.568 g=0.962
>195, 17/39, d1=0.666, d2=0.569 g=0.937
>195, 18/39, d1=0.617, d2=0.578 g=0.945
>195, 19/39, d1=0.633, d2=0.577 g=0.920
>195, 20/39, d1=0.607, d2=0.601 g=0.887
>195, 21/39, d1=0.574, d2=0.672 g=0.903
>195, 22/39, d1=0.655, d2=0.646 g=0.909
>195, 23/39, d1=0.646, d2=0.611 g=0.872
>195, 24/39, d1=0.528, d2=0.718 g=0.911
>195, 25/

>200, 13/39, d1=0.567, d2=0.546 g=0.951
>200, 14/39, d1=0.653, d2=0.612 g=0.952
>200, 15/39, d1=0.675, d2=0.565 g=0.917
>200, 16/39, d1=0.533, d2=0.675 g=0.971
>200, 17/39, d1=0.564, d2=0.584 g=1.044
>200, 18/39, d1=0.562, d2=0.572 g=0.987
>200, 19/39, d1=0.628, d2=0.654 g=1.042
>200, 20/39, d1=0.691, d2=0.612 g=0.979
>200, 21/39, d1=0.663, d2=0.566 g=0.984
>200, 22/39, d1=0.707, d2=0.583 g=0.994
>200, 23/39, d1=0.654, d2=0.610 g=0.949
>200, 24/39, d1=0.673, d2=0.632 g=0.933
>200, 25/39, d1=0.702, d2=0.594 g=0.940
>200, 26/39, d1=0.653, d2=0.616 g=0.919
>200, 27/39, d1=0.658, d2=0.616 g=0.912
>200, 28/39, d1=0.672, d2=0.670 g=0.915
>200, 29/39, d1=0.619, d2=0.605 g=0.960
>200, 30/39, d1=0.641, d2=0.608 g=0.972
>200, 31/39, d1=0.638, d2=0.638 g=1.009
>200, 32/39, d1=0.585, d2=0.606 g=1.026
>200, 33/39, d1=0.683, d2=0.699 g=0.956
>200, 34/39, d1=0.682, d2=0.613 g=0.933
>200, 35/39, d1=0.648, d2=0.580 g=0.901
>200, 36/39, d1=0.643, d2=0.625 g=0.949
>200, 37/39, d1=0.629, d2=0.600 g=0.972


In [5]:
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/39, d1=0.643, d2=0.566 g=1.013
>1, 2/39, d1=0.640, d2=0.582 g=1.043
>1, 3/39, d1=0.640, d2=0.567 g=0.974
>1, 4/39, d1=0.560, d2=0.591 g=0.958
>1, 5/39, d1=0.594, d2=0.637 g=0.959
>1, 6/39, d1=0.647, d2=0.615 g=0.969
>1, 7/39, d1=0.642, d2=0.597 g=0.988
>1, 8/39, d1=0.652, d2=0.662 g=0.951
>1, 9/39, d1=0.561, d2=0.587 g=0.990
>1, 10/39, d1=0.697, d2=0.627 g=0.929
>1, 11/39, d1=0.637, d2=0.604 g=0.895
>1, 12/39, d1=0.591, d2=0.643 g=0.874
>1, 13/39, d1=0.558, d2=0.660 g=0.938
>1, 14/39, d1=0.634, d2=0.689 g=0.917
>1, 15/39, d1=0.558, d2=0.636 g=0.990
>1, 16/39, d1=0.652, d2=0.582 g=0.923
>1, 17/39, d1=0.607, d2=0.653 g=0.980
>1, 18/39, d1=0.638, d2=0.644 g=1.025
>1, 19/39, d1=0.672, d2=0.593 g=0.998
>1, 20/39, d1=0.690, d2=0.568 g=1.035
>1, 21/39, d1=0.685, d2=0.531 g=1.064
>1, 22/39, d1=0.666, d2=0.542 g=1.007
>1, 23/39, d1=0.662, d2=0.534 g=0.992
>1, 24/39, d1=0.593, d2=0.593 g=0.938
>1, 25/39, d1=0.675, d2=0.627 g=0.926
>1, 26/39, d1=0.649, d2=0.679 g=0.940
>1, 27/39, d1=0.647, 

>6, 24/39, d1=0.554, d2=0.582 g=0.996
>6, 25/39, d1=0.574, d2=0.552 g=0.999
>6, 26/39, d1=0.655, d2=0.549 g=0.983
>6, 27/39, d1=0.605, d2=0.568 g=1.001
>6, 28/39, d1=0.621, d2=0.628 g=0.955
>6, 29/39, d1=0.636, d2=0.580 g=0.916
>6, 30/39, d1=0.484, d2=0.688 g=0.949
>6, 31/39, d1=0.493, d2=0.625 g=0.934
>6, 32/39, d1=0.511, d2=0.772 g=0.936
>6, 33/39, d1=0.535, d2=0.660 g=0.981
>6, 34/39, d1=0.657, d2=0.584 g=0.907
>6, 35/39, d1=0.675, d2=0.689 g=0.994
>6, 36/39, d1=0.606, d2=0.592 g=0.953
>6, 37/39, d1=0.608, d2=0.599 g=0.924
>6, 38/39, d1=0.607, d2=0.620 g=0.934
>6, 39/39, d1=0.771, d2=0.619 g=0.989
>7, 1/39, d1=0.626, d2=0.592 g=0.940
>7, 2/39, d1=0.563, d2=0.640 g=0.998
>7, 3/39, d1=0.767, d2=0.524 g=1.024
>7, 4/39, d1=0.633, d2=0.608 g=0.994
>7, 5/39, d1=0.608, d2=0.584 g=0.962
>7, 6/39, d1=0.624, d2=0.563 g=0.928
>7, 7/39, d1=0.613, d2=0.609 g=0.924
>7, 8/39, d1=0.663, d2=0.553 g=0.979
>7, 9/39, d1=0.631, d2=0.597 g=1.009
>7, 10/39, d1=0.655, d2=0.615 g=0.998
>7, 11/39, d1=0.640, 

>12, 3/39, d1=0.646, d2=0.657 g=0.994
>12, 4/39, d1=0.642, d2=0.576 g=0.985
>12, 5/39, d1=0.689, d2=0.531 g=1.028
>12, 6/39, d1=0.702, d2=0.524 g=1.007
>12, 7/39, d1=0.610, d2=0.561 g=0.943
>12, 8/39, d1=0.646, d2=0.573 g=0.895
>12, 9/39, d1=0.581, d2=0.625 g=0.969
>12, 10/39, d1=0.663, d2=0.635 g=0.921
>12, 11/39, d1=0.539, d2=0.662 g=0.911
>12, 12/39, d1=0.497, d2=0.628 g=0.956
>12, 13/39, d1=0.587, d2=0.661 g=0.961
>12, 14/39, d1=0.689, d2=0.618 g=0.941
>12, 15/39, d1=0.630, d2=0.578 g=0.950
>12, 16/39, d1=0.667, d2=0.632 g=0.944
>12, 17/39, d1=0.607, d2=0.590 g=1.058
>12, 18/39, d1=0.614, d2=0.532 g=1.013
>12, 19/39, d1=0.604, d2=0.593 g=1.055
>12, 20/39, d1=0.558, d2=0.522 g=1.025
>12, 21/39, d1=0.578, d2=0.579 g=0.998
>12, 22/39, d1=0.557, d2=0.570 g=0.959
>12, 23/39, d1=0.682, d2=0.653 g=0.930
>12, 24/39, d1=0.498, d2=0.771 g=0.949
>12, 25/39, d1=0.677, d2=0.610 g=0.950
>12, 26/39, d1=0.624, d2=0.584 g=1.036
>12, 27/39, d1=0.659, d2=0.563 g=1.005
>12, 28/39, d1=0.694, d2=0.638 g

>17, 21/39, d1=0.720, d2=0.795 g=1.026
>17, 22/39, d1=0.633, d2=0.706 g=1.008
>17, 23/39, d1=0.697, d2=0.553 g=1.056
>17, 24/39, d1=0.688, d2=0.590 g=0.986
>17, 25/39, d1=0.718, d2=0.495 g=0.989
>17, 26/39, d1=0.738, d2=0.562 g=1.000
>17, 27/39, d1=0.581, d2=0.609 g=0.963
>17, 28/39, d1=0.561, d2=0.607 g=0.989
>17, 29/39, d1=0.581, d2=0.634 g=0.984
>17, 30/39, d1=0.614, d2=0.588 g=0.960
>17, 31/39, d1=0.616, d2=0.538 g=0.973
>17, 32/39, d1=0.642, d2=0.619 g=0.936
>17, 33/39, d1=0.626, d2=0.669 g=0.969
>17, 34/39, d1=0.545, d2=0.617 g=0.944
>17, 35/39, d1=0.630, d2=0.572 g=1.008
>17, 36/39, d1=0.678, d2=0.590 g=0.947
>17, 37/39, d1=0.574, d2=0.569 g=1.019
>17, 38/39, d1=0.645, d2=0.538 g=1.015
>17, 39/39, d1=0.581, d2=0.632 g=0.990
>18, 1/39, d1=0.567, d2=0.605 g=0.964
>18, 2/39, d1=0.652, d2=0.619 g=1.037
>18, 3/39, d1=0.644, d2=0.611 g=0.990
>18, 4/39, d1=0.683, d2=0.575 g=1.052
>18, 5/39, d1=0.603, d2=0.580 g=0.990
>18, 6/39, d1=0.630, d2=0.543 g=1.030
>18, 7/39, d1=0.636, d2=0.615 g

>22, 37/39, d1=0.613, d2=0.560 g=0.983
>22, 38/39, d1=0.595, d2=0.626 g=0.962
>22, 39/39, d1=0.692, d2=0.597 g=0.949
>23, 1/39, d1=0.652, d2=0.624 g=0.948
>23, 2/39, d1=0.591, d2=0.701 g=0.959
>23, 3/39, d1=0.656, d2=0.589 g=0.918
>23, 4/39, d1=0.644, d2=0.579 g=0.991
>23, 5/39, d1=0.647, d2=0.580 g=0.942
>23, 6/39, d1=0.572, d2=0.618 g=0.992
>23, 7/39, d1=0.591, d2=0.600 g=1.036
>23, 8/39, d1=0.631, d2=0.657 g=0.995
>23, 9/39, d1=0.602, d2=0.568 g=0.988
>23, 10/39, d1=0.623, d2=0.723 g=1.062
>23, 11/39, d1=0.679, d2=0.541 g=0.942
>23, 12/39, d1=0.649, d2=0.630 g=0.990
>23, 13/39, d1=0.650, d2=0.611 g=1.066
>23, 14/39, d1=0.657, d2=0.566 g=1.011
>23, 15/39, d1=0.638, d2=0.558 g=0.965
>23, 16/39, d1=0.675, d2=0.554 g=0.995
>23, 17/39, d1=0.612, d2=0.621 g=1.058
>23, 18/39, d1=0.686, d2=0.552 g=0.962
>23, 19/39, d1=0.656, d2=0.545 g=0.943
>23, 20/39, d1=0.677, d2=0.585 g=0.959
>23, 21/39, d1=0.663, d2=0.617 g=0.892
>23, 22/39, d1=0.545, d2=0.580 g=0.889
>23, 23/39, d1=0.535, d2=0.599 g=0

...

>178, 31/39, d1=0.677, d2=0.543 g=1.308
>178, 32/39, d1=0.523, d2=0.491 g=1.204
>178, 33/39, d1=0.618, d2=0.517 g=1.219
>178, 34/39, d1=0.575, d2=0.493 g=1.203
>178, 35/39, d1=0.485, d2=0.511 g=1.213
>178, 36/39, d1=0.558, d2=0.526 g=1.189
>178, 37/39, d1=0.475, d2=0.599 g=1.245
>178, 38/39, d1=0.453, d2=0.427 g=1.195
>178, 39/39, d1=0.465, d2=0.623 g=1.173
>179, 1/39, d1=0.556, d2=0.510 g=1.331
>179, 2/39, d1=0.534, d2=0.426 g=1.152
>179, 3/39, d1=0.521, d2=0.500 g=1.222
>179, 4/39, d1=0.595, d2=0.467 g=1.177
>179, 5/39, d1=0.491, d2=0.617 g=1.261
>179, 6/39, d1=0.558, d2=0.483 g=1.312
>179, 7/39, d1=0.576, d2=0.500 g=1.344
>179, 8/39, d1=0.576, d2=0.716 g=1.157
>179, 9/39, d1=0.609, d2=0.453 g=1.270
>179, 10/39, d1=0.557, d2=0.596 g=1.253
>179, 11/39, d1=0.587, d2=0.566 g=1.200
>179, 12/39, d1=0.544, d2=0.543 g=1.213
>179, 13/39, d1=0.640, d2=0.496 g=1.178
>179, 14/39, d1=0.451, d2=0.508 g=1.282
>179, 15/39, d1=0.508, d2=0.494 g=1.200
>179, 16/39, d1=0.449, d2=0.478 g=1.225
>179, 17/

>184, 3/39, d1=0.528, d2=0.440 g=1.311
>184, 4/39, d1=0.601, d2=0.426 g=1.334
>184, 5/39, d1=0.571, d2=0.571 g=1.207
>184, 6/39, d1=0.625, d2=0.597 g=1.198
>184, 7/39, d1=0.547, d2=0.571 g=1.305
>184, 8/39, d1=0.490, d2=0.542 g=1.180
>184, 9/39, d1=0.555, d2=0.465 g=1.266
>184, 10/39, d1=0.607, d2=0.463 g=1.169
>184, 11/39, d1=0.440, d2=0.570 g=1.221
>184, 12/39, d1=0.522, d2=0.586 g=1.206
>184, 13/39, d1=0.557, d2=0.513 g=1.275
>184, 14/39, d1=0.546, d2=0.470 g=1.289
>184, 15/39, d1=0.589, d2=0.472 g=1.271
>184, 16/39, d1=0.600, d2=0.526 g=1.260
>184, 17/39, d1=0.531, d2=0.607 g=1.176
>184, 18/39, d1=0.435, d2=0.449 g=1.269
>184, 19/39, d1=0.625, d2=0.504 g=1.227
>184, 20/39, d1=0.557, d2=0.507 g=1.239
>184, 21/39, d1=0.470, d2=0.544 g=1.161
>184, 22/39, d1=0.561, d2=0.534 g=1.265
>184, 23/39, d1=0.493, d2=0.500 g=1.278
>184, 24/39, d1=0.490, d2=0.434 g=1.238
>184, 25/39, d1=0.477, d2=0.491 g=1.362
>184, 26/39, d1=0.545, d2=0.510 g=1.220
>184, 27/39, d1=0.448, d2=0.517 g=1.267
>184, 2

>189, 14/39, d1=0.547, d2=0.481 g=1.390
>189, 15/39, d1=0.478, d2=0.442 g=1.327
>189, 16/39, d1=0.466, d2=0.477 g=1.203
>189, 17/39, d1=0.436, d2=0.522 g=1.243
>189, 18/39, d1=0.600, d2=0.517 g=1.363
>189, 19/39, d1=0.498, d2=0.412 g=1.216
>189, 20/39, d1=0.396, d2=0.429 g=1.365
>189, 21/39, d1=0.544, d2=0.509 g=1.345
>189, 22/39, d1=0.541, d2=0.430 g=1.360
>189, 23/39, d1=0.511, d2=0.716 g=1.207
>189, 24/39, d1=0.501, d2=0.573 g=1.373
>189, 25/39, d1=0.581, d2=0.503 g=1.296
>189, 26/39, d1=0.618, d2=0.474 g=1.327
>189, 27/39, d1=0.525, d2=0.537 g=1.389
>189, 28/39, d1=0.540, d2=0.454 g=1.283
>189, 29/39, d1=0.589, d2=0.537 g=1.297
>189, 30/39, d1=0.589, d2=0.442 g=1.199
>189, 31/39, d1=0.422, d2=0.551 g=1.265
>189, 32/39, d1=0.506, d2=0.547 g=1.243
>189, 33/39, d1=0.459, d2=0.537 g=1.279
>189, 34/39, d1=0.629, d2=0.481 g=1.287
>189, 35/39, d1=0.475, d2=0.626 g=1.414
>189, 36/39, d1=0.569, d2=0.447 g=1.385
>189, 37/39, d1=0.678, d2=0.544 g=1.355
>189, 38/39, d1=0.598, d2=0.544 g=1.332


>194, 26/39, d1=0.596, d2=0.518 g=1.261
>194, 27/39, d1=0.489, d2=0.553 g=1.444
>194, 28/39, d1=0.570, d2=0.456 g=1.443
>194, 29/39, d1=0.529, d2=0.533 g=1.216
>194, 30/39, d1=0.565, d2=0.599 g=1.328
>194, 31/39, d1=0.623, d2=0.475 g=1.341
>194, 32/39, d1=0.457, d2=0.412 g=1.405
>194, 33/39, d1=0.449, d2=0.541 g=1.217
>194, 34/39, d1=0.414, d2=0.423 g=1.311
>194, 35/39, d1=0.476, d2=0.558 g=1.345
>194, 36/39, d1=0.495, d2=0.488 g=1.248
>194, 37/39, d1=0.545, d2=0.516 g=1.429
>194, 38/39, d1=0.600, d2=0.458 g=1.218
>194, 39/39, d1=0.616, d2=0.507 g=1.369
>195, 1/39, d1=0.555, d2=0.453 g=1.334
>195, 2/39, d1=0.522, d2=0.462 g=1.330
>195, 3/39, d1=0.510, d2=0.577 g=1.341
>195, 4/39, d1=0.493, d2=0.465 g=1.309
>195, 5/39, d1=0.502, d2=0.528 g=1.325
>195, 6/39, d1=0.485, d2=0.541 g=1.203
>195, 7/39, d1=0.501, d2=0.567 g=1.321
>195, 8/39, d1=0.558, d2=0.510 g=1.369
>195, 9/39, d1=0.600, d2=0.443 g=1.306
>195, 10/39, d1=0.415, d2=0.440 g=1.244
>195, 11/39, d1=0.589, d2=0.609 g=1.227
>195, 12/

>199, 37/39, d1=0.551, d2=0.507 g=1.383
>199, 38/39, d1=0.626, d2=0.423 g=1.420
>199, 39/39, d1=0.372, d2=0.417 g=1.347
>200, 1/39, d1=0.577, d2=0.432 g=1.369
>200, 2/39, d1=0.562, d2=0.405 g=1.184
>200, 3/39, d1=0.432, d2=0.548 g=1.187
>200, 4/39, d1=0.537, d2=0.549 g=1.271
>200, 5/39, d1=0.575, d2=0.551 g=1.322
>200, 6/39, d1=0.511, d2=0.530 g=1.330
>200, 7/39, d1=0.501, d2=0.546 g=1.326
>200, 8/39, d1=0.533, d2=0.548 g=1.431
>200, 9/39, d1=0.448, d2=0.533 g=1.367
>200, 10/39, d1=0.565, d2=0.521 g=1.470
>200, 11/39, d1=0.545, d2=0.501 g=1.382
>200, 12/39, d1=0.545, d2=0.433 g=1.291
>200, 13/39, d1=0.484, d2=0.371 g=1.261
>200, 14/39, d1=0.542, d2=0.475 g=1.273
>200, 15/39, d1=0.435, d2=0.501 g=1.291
>200, 16/39, d1=0.514, d2=0.660 g=1.308
>200, 17/39, d1=0.516, d2=0.661 g=1.385
>200, 18/39, d1=0.484, d2=0.623 g=1.401
>200, 19/39, d1=0.582, d2=0.479 g=1.341
>200, 20/39, d1=0.581, d2=0.468 g=1.309
>200, 21/39, d1=0.601, d2=0.527 g=1.248
>200, 22/39, d1=0.475, d2=0.547 g=1.371
>200, 23/